In [5]:
import numpy as np, pandas as pd, xarray as xr
from pathlib import Path
import datetime, networkx as nx, yaml, warnings, sys, logging, shutil, papermill
from disjoint_set import DisjointSet
from helper import singleglob, nxrender, Step, dict_merge, PdfWriter, DictWriter, TableWriter, TextWriter
import matplotlib.pyplot as plt
import matplotlib as mpl, seaborn as sns
from tqdm.notebook import tqdm

In [6]:
base_folder =  Path("/media/filer2/T4/Clara/Data/Luisa/")
dest_base_folder = Path("/home/julienb/Documents/Data/Luisa/")
force=False

In [7]:

in_sessions = [f.parent for f in base_folder.glob("**/metadata.yaml")]
sessions = pd.DataFrame()
sessions["in_session"] = in_sessions
sessions["dest_folder"] = [dest_base_folder / s.relative_to(base_folder).with_name(f"Session--{s.name}") for s in in_sessions]
sessions["already_done"] = sessions["dest_folder"].apply(lambda d: (d/"analysis").exists())
sessions


,in_session,dest_folder,already_done
0,/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_...,/home/julienb/Documents/Data/Luisa/Rat101/opto...,True
1,/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_...,/home/julienb/Documents/Data/Luisa/Rat101/opto...,True
2,/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_...,/home/julienb/Documents/Data/Luisa/Rat101/opto...,True
3,/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_...,/home/julienb/Documents/Data/Luisa/Rat101/opto...,True
4,/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_...,/home/julienb/Documents/Data/Luisa/Rat101/opto...,True
...,...,...,...
135,/media/filer2/T4/Clara/Data/Luisa/Rat246/opto_...,/home/julienb/Documents/Data/Luisa/Rat246/opto...,True
136,/media/filer2/T4/Clara/Data/Luisa/Rat246/opto_...,/home/julienb/Documents/Data/Luisa/Rat246/opto...,True
137,/media/filer2/T4/Clara/Data/Luisa/Rat246/opto_...,/home/julienb/Documents/Data/Luisa/Rat246/opto...,True
138,/media/filer2/T4/Clara/Data/Luisa/Rat246/opto_...,/home/julienb/Documents/Data/Luisa/Rat246/opto...,True


In [8]:
import contextlib
import os

devnull = open(os.devnull, 'w')
contextlib.redirect_stderr(devnull)

df = sessions[["in_session", "dest_folder"]] if force else sessions.loc[~sessions["already_done"], ["in_session", "dest_folder"]]
progress = tqdm(df.iterrows(), total=len(df.index))
copy_errors = []
execute_errors = []
for _, (s, dest_folder) in progress:
    progress.set_postfix_str(f"Copying {s.stem}, n_copy_err {len(copy_errors)}, n_exec_err {len(execute_errors)}")
    try:
        with contextlib.redirect_stdout(open(os.devnull,'w')):
            papermill.execute_notebook("copy_session.ipynb", "copy_session_tmp.ipynb", parameters=
                                    dict(dest_folder = str(dest_folder), session_folder=str(s)), progress_bar=False)
    except Exception as e:
        e.add_note(f"Error for preparing session {s}")
        copy_errors.append((s, e))
        
    else:
        progress.set_postfix_str(f"Executing {s.stem}, n_copy_err {len(copy_errors)}, n_exec_err {len(execute_errors)}")
        analysis_folder = dest_folder/ "analysis"
        for script in ["events.ipynb", "task_graph.ipynb"]:
            shutil.copy(script, analysis_folder/script)
        # (analysis_folder/"events").mkdir(exist_ok=True, parents=True)
        # try:
        #     papermill.execute_notebook(analysis_folder/"events.ipynb", analysis_folder/"events"/"notebook.ipynb", cwd=analysis_folder, progress_bar=False)
        # except Exception as e:
        #     execute_errors.append(s)
        #     e.add_note(f"Error for executing session {s}")
        #     print(e)
if len(copy_errors) > 0:
    print(f"Sessions with copy errors:{[s for s, e in copy_errors]}")
    raise ExceptionGroup("Errors during copy", [e for s, e in copy_errors])
dict(copy_errors = copy_errors, execute_errors=execute_errors)



  0%|          | 0/11 [00:00<?, ?it/s]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0

{'copy_errors': [(PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_CueRT/Rat101_20240909'),
   papermill.exceptions.PapermillExecutionError(7,
                                                8,
                                                'task  = metadata["task"]["name"]\ntry:\n    task_path = singleglob(session_folder, f"*{task}*.xls")\nexcept FileNotFoundError:\n    task_path = singleglob(templates_folder / "PolyEventTemplates",f"*{task}*.xls")\ntask_poly_template = singleglob(templates_folder / "PolyEventTemplates",f"*{task}*.yaml")\npoly_template = yaml.safe_load(task_poly_template.open("r"))\npoly_template\n',
                                                'FileNotFoundError',
                                                "Found 0 candidates for pattern ('*HF_55_RandomTrial_NoGoGo_left_both_sound_LaserChrim_S1_FastestMT1_4p15Hz_2rew_NoPadCheck_counter*.yaml',) in folder /home/julienb/Documents/Luisa/scripts_and_templates/PolyEventTemplates",
                        